# Data Semantics
Il dataset contiene al suo interno dei record che descrivono l'acquisto di un prodotto nel dettaglio e il suo acquirente. Inoltre ci fornisce l'informazione del carrello a cui appartiene il prodotto.

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
df = pd.read_csv("dataset/customer_supermarket.csv", sep="\t",index_col=0,decimal=",")
df.info()
print(df.head())
print("\nNumero di record")
len(df.index)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   BasketID         471910 non-null  object 
 1   BasketDate       471910 non-null  object 
 2   Sale             471910 non-null  float64
 3   CustomerID       406830 non-null  object 
 4   CustomerCountry  471910 non-null  object 
 5   ProdID           471910 non-null  object 
 6   ProdDescr        471157 non-null  object 
 7   Qta              471910 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 32.4+ MB
  BasketID      BasketDate  Sale CustomerID CustomerCountry  ProdID  \
0   536365  01/12/10 08:26  2.55    17850.0  United Kingdom  85123A   
1   536365  01/12/10 08:26  3.39    17850.0  United Kingdom   71053   
2   536365  01/12/10 08:26  2.75    17850.0  United Kingdom  84406B   
3   536365  01/12/10 08:26  3.39    17850.0  United Kingdom  84029G   
4 

471910

Il dataset preso in analisi è composto da 471.910 record descritto da 8 attributi

* BasketID: (Nominale) ID del carrello in cui il prodotto è contenuto
* BasketDate: (Numerico, intervallo) La data in cui è avvenuto l'acquisto
* Sale: (Numerico, continuo???) Il prezzo pagato per l'acquisto dal cliente. Dominio N>0
* CustomerID: (nominale) ID del cliente. Dominio N
* CustomerCountry: (Nominale) La città di appartenenza del cliente
* ProdID: (Nominale) ID del prodotto acquistato
* ProdDescr: (Nominale) Descrizione del prodotto
* Qta: (Numerico, discreto) Quantità di prodotto acquistata. Dominio N>0

## Variable transformation
In questo notebook si guarda il tipo dei vari attributi che abbiamo e si modificano eventualmente

### BasketID ✔️
Cerchiamo di capire il tipo di dato. Vediamo che è composto da Lettere maiuscole e numeri.

In [54]:
print("MAIUSC: "+str(df.BasketID.str.contains('[A-Z]').any()))
print("minuscole: "+str(df.BasketID.str.contains('[a-z]').any()))
print("Numeri: "+str(df.BasketID.str.contains('[0-9]').any()))
print("Segni speciali: "+str(df.BasketID.str.contains('[^A-Za-z0-9]').any()))

MAIUSC: True
minuscole: False
Numeri: True
Segni speciali: False


è corretto mantenerlo come letterale

### ProdID ✔️
è un oggetto vediamo come è composto

In [56]:
print("MAIUSC: "+str(df.ProdID.str.contains('[A-Z]').any()))
print("minuscole: "+str(df.ProdID.str.contains('[a-z]').any()))
print("Numeri: "+str(df.ProdID.str.contains('[0-9]').any()))
print("Segni speciali: "+str(df.ProdID.str.contains('[^A-Za-z0-9]').any()))

MAIUSC: True
minuscole: True
Numeri: True
Segni speciali: True


In [58]:
import re
regexp = re.compile(r'[^A-Za-z0-9]')
dict_special={}
for index, row in df["ProdID"].iteritems():
    if(regexp.search(row)):
        if(row in dict_special):
            dict_special[row]+=1
        else:
            dict_special[row]=1
print(dict_special)

{'BANK CHARGES': 25, 'gift_0001_40': 3, 'gift_0001_30': 5, 'gift_0001_20': 4, 'gift_0001_10': 4, 'gift_0001_50': 2}


In particolare i prodotti che contengono caratteri speciali sono pochi e sono di fue categorie BANK CHARGES e gift. In ogni caso va bene mantenerlo come stringa

### ProdDescr  ✔️
contiene la descrizione dei prodotti, è un attributo nominale e non c'è bisogno di nessuna conversione

### Qta ✔️
Indica la quantità acquistata di un determinato prodotto, in questo caso abbiamo degli interi ed è giusto che sia così

### BasketDate ⚠️
Dovrebbe essere un datetime ma in realtà è un ogetto, provvediamo a modificarlo

In [60]:
df["BasketDate"] = pd.to_datetime(df['BasketDate'], format='%d/%m/%y %H:%M')

### CustomerID ⚠️✔
Questo campo è un float ma potrebbe essere tranquillamente una stringa. Controlliamo che non ci siano decimali, in modo da convertire prima a int e poi a stringa

In [62]:
'''import math
count=0
for index, row in df["CustomerID"].iteritems():
    if(row %1!=0.0 and not math.isnan(row)):
        count+=1
        print(row)
print(count)'''
        

'import math\ncount=0\nfor index, row in df["CustomerID"].iteritems():\n    if(row %1!=0.0 and not math.isnan(row)):\n        count+=1\n        print(row)\nprint(count)'

Trasformiamo tutto in stringa e eliminiamo la parte decimale per pulizia. Preservando i valori che sono a null che verranno trattati in seguito.

In [64]:
len(df[df["CustomerID"].isnull()])

65080

In [66]:
#df["CustomerID"] = df['CustomerID'].astype(str).str.replace('\.0', '')

In [68]:
#len(df[df["CustomerID"]=="nan"])

### Salvataggio dataframe
Salviamo il dataframe con i tipi di dato modificati

In [70]:

df.BasketID = df.BasketID.astype(str)
df.CustomerID = df.CustomerID.astype(str)
df.CustomerCountry = df.CustomerCountry.astype(str)
df.ProdID = df.ProdID.astype(str)
df.ProdDescr = df.ProdDescr.astype(str)
df.to_csv('dataset/TC-dataset.csv', sep='\t',decimal=",")

In [ ]:
df = pd.read_csv("dataset/TC-dataset.csv", sep="\t",index_col=0,decimal=",")
df.info()